<a href="https://colab.research.google.com/github/enliktjioe/nlp2022/blob/master/week1/day3_pytorch_sentiment_analysis_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reference

- https://github.com/bentrevett/pytorch-sentiment-analysis
- pytorch==1.8.0 
- torchtext==0.9.0
- torchvision==0.9.0
- https://discuss.pytorch.org/t/cuda-error-cublas-status-internal-error-when-calling-cublascreate-handle/114341/11?u=confifu

# Simple Sentiment Analysis

In [1]:
#!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
#!pip install torchtext==0.9.0

In [3]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [4]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [5]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [6]:
print(vars(train_data.examples[0]))

{'text': ['I', 'did', "n't", 'expect', 'Val', 'Kilmer', 'to', 'make', 'a', 'convincing', 'John', 'Holmes', ',', 'but', 'I', 'found', 'myself', 'forgetting', 'that', 'it', 'was', "n't", 'the', 'porn', 'legend', 'himself', '.', 'In', 'fact', ',', 'the', 'entire', 'cast', 'turned', 'in', 'amazing', 'performances', 'in', 'this', 'vastly', 'under', '-', 'rated', 'movie.<br', '/><br', '/>As', 'some', 'have', 'mentioned', 'earlier', ',', 'seek', 'out', 'the', 'two', '-', 'disc', 'set', 'and', 'watch', 'the', '"', 'Wadd', '"', 'documentary', 'first', ';', 'it', 'will', 'give', 'you', 'a', 'lot', 'of', 'background', 'on', 'the', 'story', 'which', 'will', 'be', 'helpful', 'in', 'appreciating', 'the', 'movie', '.', '<', 'br', '/><br', '/>Some', 'people', 'seem', 'unhappy', 'about', 'the', 'LAPD', 'crime', 'scene', 'video', 'being', 'included', 'on', 'the', 'DVD', '.', 'There', 'are', 'a', 'number', 'of', 'reasons', 'that', 'it', 'might', 'have', 'been', 'included', ',', 'one', 'of', 'which', 'is'

In [7]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [9]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [10]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 201514), (',', 190418), ('.', 165254), ('a', 108907), ('and', 108610), ('of', 100111), ('to', 93339), ('is', 75916), ('in', 60951), ('I', 54167), ('it', 53530), ('that', 48932), ('"', 43884), ("'s", 42974), ('this', 42421), ('-', 36744), ('/><br', 35492), ('was', 35212), ('as', 30316), ('movie', 29892)]


In [12]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [13]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [14]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [15]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [16]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


## Train the Model

In [18]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [19]:
criterion = nn.BCEWithLogitsLoss()

In [20]:
model = model.to(device)
criterion = criterion.to(device)

In [21]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [22]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 16s
	Train Loss: 0.694 | Train Acc: 50.15%
	 Val. Loss: 0.697 |  Val. Acc: 49.62%
Epoch: 02 | Epoch Time: 0m 15s
	Train Loss: 0.693 | Train Acc: 49.81%
	 Val. Loss: 0.697 |  Val. Acc: 49.00%
Epoch: 03 | Epoch Time: 0m 15s
	Train Loss: 0.693 | Train Acc: 50.18%
	 Val. Loss: 0.698 |  Val. Acc: 50.74%
Epoch: 04 | Epoch Time: 0m 15s
	Train Loss: 0.693 | Train Acc: 49.53%
	 Val. Loss: 0.697 |  Val. Acc: 48.84%
Epoch: 05 | Epoch Time: 0m 15s
	Train Loss: 0.693 | Train Acc: 50.04%
	 Val. Loss: 0.698 |  Val. Acc: 50.71%


In [26]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.710 | Test Acc: 47.64%


## Next Steps

In the next notebook, the improvements we will make are:

- packed padded sequences
- pre-trained word embeddings
- different RNN architecture
- bidirectional RNN
- multi-layer RNN
- regularization
- a different optimizer

This will allow us to achieve ~84% accuracy.